- Env: py-39-4: 能够完整运行  --jit --ipex
    - innersource-ipex ( 174a1874adf ）: 2.1.0+git174a187
    - sandbox-transformers ( 1720855 ) : 4.28.1
```
Inference latency: 6.855 sec.
First token average latency: 0.681 sec.
Average 2... latency: 0.199 sec.
P90 2... latency: 0.295 sec.
P99 2... latency: 1.128 sec.
====================
prim_kind      impl          ncalls  time(ms)   overall%  agg_ncalls(avg)  agg_time(ms)  agg_overall%
inner_product  x64:gemm:jit  81965   123431.87  99.53     81965.00         123431.87     99.53
reorder        jit:uni       169     587.51     0.47      41067.00         124019.38     100.00
```

In [2]:
!ONEDNN_VERBOSE=1 python run_generation.py --device cpu --input-tokens=32 --max-new-tokens 32 \
    --num-iter=15 --num-warmup=5 --batch-size=1 -m EleutherAI/gpt-j-6b \
    --jit --ipex --token-latency --dtype bfloat16 > /tmp/gptj.log 2>/dev/null

In [6]:
!tail -n 5 /tmp/gptj.log
!echo ====================
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/gptj.log -g breakdown -k prim_kind  impl | column -t -s,

Inference latency: 6.855 sec.
First token average latency: 0.681 sec.
Average 2... latency: 0.199 sec.
P90 2... latency: 0.295 sec.
P99 2... latency: 1.128 sec.


prim_kind      impl          ncalls  time(ms)   overall%  agg_ncalls(avg)  agg_time(ms)  agg_overall%
inner_product  x64:gemm:jit  81965   123431.87  99.53     81965.00         123431.87     99.53
reorder        jit:uni       169     587.51     0.47      41067.00         124019.38     100.00


In [1]:
!ONEDNN_VERBOSE=1 python run_generation-2.py --device cpu --input-tokens=32 --max-new-tokens 32 \
    --num-iter=15 --num-warmup=5 --batch-size=1 -m EleutherAI/gpt-j-6b \
    --jit --ipex --token-latency --dtype bfloat16 --profile > /tmp/gptj-prof_1.log 2>/dev/null

In [4]:
!tail -n 25 /tmp/gptj-prof_1.log 
!echo ====================
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/gptj-prof_1.log  -g breakdown -k prim_kind  impl | column -t -s,

['Once upon a time, there existed a little girl, who liked to have adventures. She wanted to go to places and meet new people, and have fun. One day, she decided to go on an adventure. She packed her bags, and set off on her journey.\n\nThe little girl walked and walked,'] [32]
Iteration: 14, Time: 5.689841 sec

 ---------- Summary: ----------
Inference latency: 5.653 sec.
First token average latency: 0.484 sec.
Average 2... latency: 0.167 sec.
P90 2... latency: 0.170 sec.
P99 2... latency: 0.192 sec.
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                               ipex_prepack::linear_run        51.3

prim_kind      impl          ncalls  time(ms)   overall%  agg_ncalls(avg)  agg_time(ms)  agg_overall%
inner_product  x64:gemm:jit  81965   113372.98  99.29     81965.00         113372.98     99.29
reorder        jit:uni       169     816.02     0.71      41067.00         114189.01     100.00
